In [19]:
import os
import time
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

name = 'Gerrit Cole'
options = Options()
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')

driver_path = '/usr/local/bin/chromedriver'  # 請換成本機實際路徑
service = Service()
driver = webdriver.Chrome(service=service, options=options)

# 設定影片儲存資料夾 ../raw_video/Gerrit Cole/
output_dir = os.path.join('../raw_video', name)
os.makedirs(output_dir, exist_ok=True)

# 訪問 MLB Statcast 頁面
url = "https://baseballsavant.mlb.com/statcast_search?hfPT=FF%7C&hfAB=&hfGT=R%7C&hfPR=&hfZ=&hfStadium=&hfBBL=&hfNewZones=&hfPull=&hfC=&hfSea=2023%7C&hfSit=&player_type=pitcher&hfOuts=&hfOpponent=&pitcher_throws=&batter_stands=&hfSA=&game_date_gt=&game_date_lt=&hfMo=&hfTeam=&home_road=&hfRO=&position=&hfInfield=&hfOutfield=&hfInn=&hfBBT=&hfFlag=&pitchers_lookup%5B%5D=543037&metric_1=&group_by=name&min_pitches=0&min_results=0&min_pas=0&sort_col=pitches&player_event_sort=api_p_release_speed&sort_order=desc&type=details&player_id=543037"
driver.get(url)

wait = WebDriverWait(driver, 20)

video_links = []

try:
    # 等待表格出現
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#ajaxTable_543037 tbody tr")))

    rows = driver.find_elements(By.CSS_SELECTOR, "#ajaxTable_543037 tbody tr")
    print(f"總共有 {len(rows)} 列資料")

    for idx, row in enumerate(rows[:3]):
        try:
            link_element = row.find_element(By.CSS_SELECTOR, "td:nth-child(15) > a")
            href = link_element.get_attribute("href")
            print(f"[{idx+1}] 抓到影片頁面連結: {href}")
            
            driver.execute_script("window.open(arguments[0]);", href)
            driver.switch_to.window(driver.window_handles[1])

            source = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "video#sporty > source"))
            )
            video_url = source.get_attribute("src")
            print(f"  → 影片檔案連結: {video_url}")
            video_links.append(video_url)

            # 下載影片
            response = requests.get(video_url, stream=True)
            if response.status_code == 200:
                filename = f"{name} {idx+1}.mp4"
                filepath = os.path.join(output_dir, filename)
                with open(filepath, 'wb') as f:
                    for chunk in response.iter_content(chunk_size=8192):
                        f.write(chunk)
                print(f"  → 成功下載: {filename}")
            else:
                print(f"  → 無法下載影片，狀態碼: {response.status_code}")

            driver.close()
            driver.switch_to.window(driver.window_handles[0])

        except Exception as inner_e:
            print(f"[{idx+1}] 無法取得影片頁面或影片來源: {inner_e}")
            if len(driver.window_handles) > 1:
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
        time.sleep(1)  # 避免封鎖

except Exception as e:
    print("錯誤：", e)

driver.quit()
print(f"\n✅ 抓取完畢，共獲得影片數量：{len(video_links)}，已儲存至：{output_dir}")


總共有 1737 列資料
[1] 抓到影片頁面連結: https://baseballsavant.mlb.com/sporty-videos?playId=8efb7f8d-5a97-4ef8-abf0-ac7c60d0b24a
  → 影片檔案連結: https://sporty-clips.mlb.com/Yms2ODlfWGw0TUFRPT1fRGxRSEJsTlNYbFlBRFFNTFVRQUFWRkJlQUFCWEJWRUFVd1lDVXdVRlZRTUhBZ1lI.mp4
  → 成功下載: Gerrit Cole 1.mp4
[2] 抓到影片頁面連結: https://baseballsavant.mlb.com/sporty-videos?playId=a2ce108c-c3ed-4863-b76e-59243aed729f
  → 影片檔案連結: https://sporty-clips.mlb.com/Yms2ODlfWGw0TUFRPT1fVndNQ0FWVUVYbEVBVzFGWEFnQUFDUUJWQUFNQ1ZRUUFCMXdIQkFCVVZGZFNBZ3NB.mp4
  → 成功下載: Gerrit Cole 2.mp4
[3] 抓到影片頁面連結: https://baseballsavant.mlb.com/sporty-videos?playId=96b63622-7e08-4e35-a640-6379c23e25f5
  → 影片檔案連結: https://sporty-clips.mlb.com/Yms2ODlfWGw0TUFRPT1fRHdjRFVsY0NWQUFBRHdjQ1hnQUFWQVZUQUFBRFYxRUFCRllDQ1ZBSEFsWlhCVlJU.mp4
  → 成功下載: Gerrit Cole 3.mp4

✅ 抓取完畢，共獲得影片數量：3，已儲存至：../raw_video\Gerrit Cole
